In [1]:
import pandas as pd

In [2]:
df = pd.read_csv (r'E:/PhD/16-09-23.csv') 

In [3]:
print(df)

        Packet ID        TIME  Size            eth.src            eth.dst  \
0               1  1474552802    70  18:b7:9e:02:20:44  14:cc:20:51:33:ea   
1               2  1474552802    66  18:b7:9e:02:20:44  14:cc:20:51:33:ea   
2               3  1474552802    66  14:cc:20:51:33:ea  30:8c:fb:2f:e4:b2   
3               4  1474552802    60  d0:52:a8:00:67:5e  14:cc:20:51:33:ea   
4               6  1474552802    88  70:ee:50:18:34:43  14:cc:20:51:33:ea   
...           ...         ...   ...                ...                ...   
802576     947066  1474639198    95  44:65:0d:56:cc:d3  14:cc:20:51:33:ea   
802577     947067  1474639198    66  14:cc:20:51:33:ea  30:8c:fb:2f:e4:b2   
802578     947068  1474639199    54  14:cc:20:51:33:ea  44:65:0d:56:cc:d3   
802579     947071  1474639199   156  30:8c:fb:2f:e4:b2  14:cc:20:51:33:ea   
802580     947072  1474639199    66  14:cc:20:51:33:ea  30:8c:fb:2f:e4:b2   

                IP.src          IP.dst  IP.proto  port.src  port.dst  
0   

In [4]:
columns=df.columns

In [11]:
columns[0]

'Packet ID'

In [1]:
import json
import pandas as pd
pd.set_option('display.max_rows', None)
import os
import numpy as np
import csv
import random

In [2]:
os.chdir("E:/PhD/json")
files=os.listdir('E:/PhD/json/')
data=[]
for file in files:    
    with open(file) as f:
        df2 = json.load(f)
        data.append(df2)

In [14]:
df.head()

,ietf-mud:mud,ietf-access-control-list:access-lists
mud-version,1,NaN
mud-url,https://amazonecho.com/amazonecho,NaN
last-update,2018-09-16T13:53:04.908+10:00,NaN
cache-validity,100,NaN
is-supported,True,NaN


In [3]:
def getSourceDestination(data):
    if 'dnsname' in str(data):
        src_net_key = 'ietf-acldns:src-dnsname'
        dest_net_key = 'ietf-acldns:dst-dnsname'
    elif 'network' in str(data):
        src_net_key = 'source-ipv4-network'
        dest_net_key = 'destination-ipv4-network'
    else:
        src_net_key = '*'
        dest_net_key = '*'
    return data.get(src_net_key, '*'), data.get(dest_net_key, '*'), data.get('protocol', '*')

In [4]:
final_list = []
for i in range(len(data)):
    access_json=data[i].get('ietf-access-control-list:access-lists').get('acl')    
    for acl in access_json:
        aces = acl['aces']['ace']
        for ace in aces:
            final_row = {
            'sEth': '*',
            'dEth': '*',
            'typEth': '*',
            }
            if 'ethertype' in str(ace['matches']):
                final_row['sEth'] = ace['matches']['eth'].get('source-mac-address', '*')
                final_row['dEth'] = ace['matches']['eth'].get('destination-mac-address', '*')
                final_row['typEth'] = ace['matches']['eth'].get('ethertype', '*')
            if 'ipv4' in str(ace['matches']):
                source, dest, proto = getSourceDestination(ace['matches']['ipv4'])
                final_row['Source'] = source
                final_row['Destination'] = dest
                final_row['proto'] = proto
            elif 'ipv6' in str(ace['matches']):
                source, dest, proto = getSourceDestination(ace['matches']['ipv6'])
                final_row['Source'] = source
                final_row['Destination'] = dest
                final_row['proto'] = proto
            else:
                final_row['Source'] = '*'
                final_row['Destination'] = '*'
                final_row['proto'] = '*'
            if 'ipv4' in str(ace['matches']):
                source, dest, proto = getSourceDestination(ace['matches']['ipv4'])
                if proto in [1,6,17] :
                    final_row['typEth']='0x0800'
            if 'icmp' in str(ace['matches']):
                final_row['type'] = ace['matches']['icmp'].get('type', '*')
                final_row['code'] = ace['matches']['icmp'].get('code', '*')
            else:
                final_row['type'] = '*' 
                final_row['code'] = '*'   
            if 'source-port' in str(ace['matches']):
                final_row['sPort'] = ace['matches']['udp']['source-port'].get('port', '*') if 'udp' in str(ace['matches']) else ace['matches']['tcp']['source-port'].get('port', '*')
            else:
                final_row['sPort'] = '*'
            if 'destination-port' in str(ace['matches']):
                final_row['dPort'] = ace['matches']['udp']['destination-port'].get('port', '*') if 'udp' in str(ace['matches']) else ace['matches']['tcp']['destination-port'].get('port', '*')
            else:
                final_row['dPort'] = '*'
            final_row['priority'] = '*'
            final_row['action'] = 'forward' if ace['actions'].get('forwarding', '') == 'accept' else '*'
            if final_row['Source'] == '*' and final_row['Destination'] != '*':
                final_row['sEth'] = '9e:8d:de:80:29:28'  
            if final_row['Destination'] == '*' and final_row['Source'] != '*':
                final_row['dEth'] = '9e:8d:de:80:29:28'     
            final_list.append(final_row)

df2 = pd.DataFrame(final_list)
df2.head()

,sEth,dEth,typEth,Source,Destination,proto,type,code,sPort,dPort,priority,action
0,9e:8d:de:80:29:28,*,0x0800,*,dcape-na.amazon.com,6,*,*,*,443,*,forward
1,9e:8d:de:80:29:28,*,0x0800,*,softwareupdates.amazon.com,6,*,*,*,443,*,forward
2,9e:8d:de:80:29:28,*,0x0800,*,3.north-america.pool.ntp.org,17,*,*,*,123,*,forward
3,9e:8d:de:80:29:28,*,*,*,224.0.0.22/32,2,*,*,*,*,*,forward
4,9e:8d:de:80:29:28,*,0x0800,*,239.255.255.250/32,17,*,*,*,1900,*,forward


In [29]:
len(df)

57

In [38]:
len(df1)

65

In [43]:
len(df2)

15

In [47]:
len(df3)

43

In [51]:
len(df4)

9

In [55]:
len(df5)

29

In [59]:
len(df6)

201

In [63]:
len(df7)

19

In [67]:
len(df8)

13

In [71]:
len(df9)

42

In [75]:
len(df10)

38

In [79]:
len(df11)

17

In [83]:
len(df12)

18

In [87]:
len(df13)

104

In [91]:
len(df14)

48

In [95]:
len(df15)

9

In [99]:
len(df16)

17

In [103]:
len(df17)

17

In [107]:
len(df18)

39

In [111]:
len(df19)

14

In [115]:
len(df20)

33

In [119]:
len(df21)

32

In [123]:
len(df22)

43

In [127]:
len(df23)

31

In [131]:
len(df24)

33

In [135]:
len(df25)

19

In [139]:
len(df26)

9

In [143]:
len(df27)

11

In [ ]:
total number of rules-1025

In [6]:
len(df29)

1025

In [13]:
df1.to_csv('E:/PhD/json/amazonecho.csv')

In [11]:
len(df1)

57

In [14]:
df1.shape

(57, 12)

In [19]:
len(df2)

65

In [5]:
df2.to_csv('E:/PhD/json/all_device.csv')

In [21]:
df2.shape

(65, 12)

In [6]:
raw_df = pd.read_csv("E:/PhD/json/all_device.csv")

In [7]:
raw_df.head()

,Unnamed: 0,sEth,dEth,typEth,Source,Destination,proto,type,code,sPort,dPort,priority,action
0,0,9e:8d:de:80:29:28,*,0x0800,*,dcape-na.amazon.com,6,*,*,*,443,*,forward
1,1,9e:8d:de:80:29:28,*,0x0800,*,softwareupdates.amazon.com,6,*,*,*,443,*,forward
2,2,9e:8d:de:80:29:28,*,0x0800,*,3.north-america.pool.ntp.org,17,*,*,*,123,*,forward
3,3,9e:8d:de:80:29:28,*,*,*,224.0.0.22/32,2,*,*,*,*,*,forward
4,4,9e:8d:de:80:29:28,*,0x0800,*,239.255.255.250/32,17,*,*,*,1900,*,forward


In [8]:
raw_df.drop('type', inplace=True, axis=1)

In [9]:
raw_df.drop('code', inplace=True, axis=1)

In [10]:
raw_df.drop('priority', inplace=True, axis=1)

In [11]:
raw_df.head()

,Unnamed: 0,sEth,dEth,typEth,Source,Destination,proto,sPort,dPort,action
0,0,9e:8d:de:80:29:28,*,0x0800,*,dcape-na.amazon.com,6,*,443,forward
1,1,9e:8d:de:80:29:28,*,0x0800,*,softwareupdates.amazon.com,6,*,443,forward
2,2,9e:8d:de:80:29:28,*,0x0800,*,3.north-america.pool.ntp.org,17,*,123,forward
3,3,9e:8d:de:80:29:28,*,*,*,224.0.0.22/32,2,*,*,forward
4,4,9e:8d:de:80:29:28,*,0x0800,*,239.255.255.250/32,17,*,1900,forward


In [13]:
raw_df = raw_df[['sEth','dEth','typEth','proto','sPort','dPort','Source','Destination','action']]

In [14]:
raw_df.head()

,sEth,dEth,typEth,proto,sPort,dPort,Source,Destination,action
0,9e:8d:de:80:29:28,*,0x0800,6,*,443,*,dcape-na.amazon.com,forward
1,9e:8d:de:80:29:28,*,0x0800,6,*,443,*,softwareupdates.amazon.com,forward
2,9e:8d:de:80:29:28,*,0x0800,17,*,123,*,3.north-america.pool.ntp.org,forward
3,9e:8d:de:80:29:28,*,*,2,*,*,*,224.0.0.22/32,forward
4,9e:8d:de:80:29:28,*,0x0800,17,*,1900,*,239.255.255.250/32,forward


In [15]:
raw_df.to_csv('E:/PhD/json/all_device1.csv')